#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 19)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,21,B,SMPC,"7,500",14.00,1,3L,SET100,2022-02-02


In [3]:
orders_log = pd.read_csv('../data/orders-log.csv')
orders_log['trans'] = orders_log.apply(lambda row: categorise(row), axis=1)
orders_log['amount'] = orders_log.qty * orders_log.target
orders_log[orders_log.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,BDMS,-7,1L,SET50,4200,25.000000,26.750000,0.000000,0.00%,2,2022-03-09,Buy,105000.000000
2,B,BGC,-5,DOS,SET,10000,9.800000,10.100000,0.100000,+1.00%,2,2022-05-25,Buy,98000.000000
4,B,IVL,-16,XXX,SET50,2400,45.000000,49.000000,0.250000,+0.51%,2,2022-05-27,Buy,108000.000000
5,B,JASIF,-2,DOS,SET,10000,10.900000,11.100000,0.000000,0.00%,2,2022-05-24,Buy,109000.000000


In [6]:
orders_log.head().sort_values('spd',ascending=False).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BBL,-4,1L,SET50,"2,400",124.00,126.00,0.50,+0.40%,1,2022-04-22,Buy,"297,600.00"
3,B,HREIT,-4,RD05pct,SET,"10,000",8.20,8.40,0.10,+1.20%,1,2022-03-04,Buy,"82,000.00"
2,B,BGC,-5,DOS,SET,"10,000",9.80,10.10,0.10,+1.00%,2,2022-05-25,Buy,"98,000.00"
1,B,BDMS,-7,1L,SET50,"4,200",25.00,26.75,0.00,0.00%,2,2022-03-09,Buy,"105,000.00"
4,B,IVL,-16,XXX,SET50,"2,400",45.00,49.00,0.25,+0.51%,2,2022-05-27,Buy,"108,000.00"


### End of Tables in the process

### Print to verify before upload file

In [10]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [11]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [12]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,BDMS,"4,200",25.00,"105,000.00",1L,SET50,2,2022-03-09
1,B,BGC,"10,000",9.80,"98,000.00",DOS,SET,2,2022-05-25
2,B,IVL,"2,400",45.00,"108,000.00",XXX,SET50,2,2022-05-31
3,B,JASIF,"10,000",10.90,"109,000.00",DOS,SET,2,2022-05-24


In [13]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    420000.0
Name: amount, dtype: float64

In [14]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,BDMS,"4,200",25.00,"105,000.00",1L,SET50,2,2022-03-09
1,B,BGC,"10,000",9.80,"98,000.00",DOS,SET,2,2022-05-25
2,B,IVL,"2,400",45.00,"108,000.00",XXX,SET50,2,2022-05-31
3,B,JASIF,"10,000",10.90,"109,000.00",DOS,SET,2,2022-05-24


In [15]:
cash = 394_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(394000, 420000.0, -26000.0)

In [16]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate


In [17]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(394000, 0.0, 394000.0)

### After call ord-log (must call everytime that orders change)

In [18]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [19]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [20]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BBL,-4,1L,SET50,2400,124.0,126.0,0.5,+0.40%,1,2022-04-22,Buy,297600.0
2,B,BGC,-5,DOS,SET,10000,9.8,10.1,0.1,+1.00%,2,2022-05-25,Buy,98000.0
3,B,HREIT,-4,RD05pct,SET,10000,8.2,8.4,0.1,+1.20%,1,2022-03-04,Buy,82000.0
5,B,JASIF,-2,DOS,SET,10000,10.9,11.1,0.0,0.00%,2,2022-05-24,Buy,109000.0


In [21]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [22]:
ms = (df.trans == 'Sell') & (df.spd <= 7)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
10,S,DOHOME,7,C20.25,SET100,2400,20.3,19.6,0.5,+2.62%,1,2022-02-02,Sell,48720.0


In [23]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BBL,-4,1L,SET50,2400,124.0,126.0,0.5,+0.40%,1,2022-04-22,Buy,297600.0
2,B,BGC,-5,DOS,SET,10000,9.8,10.1,0.1,+1.00%,2,2022-05-25,Buy,98000.0
3,B,HREIT,-4,RD05pct,SET,10000,8.2,8.4,0.1,+1.20%,1,2022-03-04,Buy,82000.0
5,B,JASIF,-2,DOS,SET,10000,10.9,11.1,0.0,0.00%,2,2022-05-24,Buy,109000.0
10,S,DOHOME,7,C20.25,SET100,2400,20.3,19.6,0.5,+2.62%,1,2022-02-02,Sell,48720.0


In [24]:
df[mb | ms].shape[0]

5

In [25]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
0,BBL,Buy,"297,600.00",-4
6,KBANK,Buy,"210,000.00",-8
7,KKP,Buy,"204,000.00",-11
5,JASIF,Buy,"109,000.00",-2
4,IVL,Buy,"108,000.00",-16


### Select source of orders between these two

In [26]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,BBL,2400,124.0,1,-4,126.0,0.5,+0.40%,1L,SET50,2022-04-22
2,Buy,BGC,10000,9.8,2,-5,10.1,0.1,+1.00%,DOS,SET,2022-05-25
3,Buy,HREIT,10000,8.2,1,-4,8.4,0.1,+1.20%,RD05pct,SET,2022-03-04
5,Buy,JASIF,10000,10.9,2,-2,11.1,0.0,0.00%,DOS,SET,2022-05-24
10,Sell,DOHOME,2400,20.3,1,7,19.6,0.5,+2.62%,C20.25,SET100,2022-02-02


In [27]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,BDMS,4200,25.0,2,-7,26.75,0.00,0.00%,1L,SET50,2022-03-09
2,Buy,BGC,10000,9.8,2,-5,10.10,0.10,+1.00%,DOS,SET,2022-05-25
4,Buy,IVL,2400,45.0,2,-16,49.00,0.25,+0.51%,XXX,SET50,2022-05-31
5,Buy,JASIF,10000,10.9,2,-2,11.10,0.00,0.00%,DOS,SET,2022-05-24


In [28]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process